In [ ]:
%%sh

docker_name=`docker ps --format '{{.Names}}'`
#echo $docker_name
docker kill $docker_name

In [ ]:
!cp ../../11-manual-model-load/dcn/model/rfcn_dcn_coco-0000.params dcn/model/

In [ ]:
#!sudo yum install -y tree
!tree dcn/model/

In [ ]:
#!pygmentize inference.py

In [ ]:
!(cd dcn/model/ && tar -czvf /tmp/model-rfcn.tar.gz *)

In [ ]:
!aws s3 cp /tmp/model-rfcn.tar.gz s3://ar54/super_models/
!aws s3 ls s3://ar54/super_models/   

In [ ]:
!aws s3 ls s3://ar54/super_models/model-rfcn.tar.gz

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=chazarey-mxnet-serving-160-gpu-py2

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
#region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

#docker build -t chazarey-mxnet-serving:1.6.0-gpu-py3 -f docker/1.6.0/py3/Dockerfile.gpu .

docker build -t ${algorithm_name} -f Dockerfile.gpu .

docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
%%time

import sagemaker

role = sagemaker.session.get_execution_role()

from sagemaker.mxnet import MXNetModel

model_data="s3://ar54/super_models/model-rfcn.tar.gz"

model = MXNetModel(
    model_data=model_data,
    role=role,
    image="308412838853.dkr.ecr.us-east-2.amazonaws.com/chazarey-mxnet-serving-160-gpu-py2:latest",
    entry_point="inference.py",
    py_version='py2',
    framework_version='1.6.0',
    enable_cloudwatch_metrics=True
)

#predictor = model.deploy(instance_type="local_gpu", initial_instance_count=1)
#predictor = model.deploy(instance_type="local", initial_instance_count=1)
predictor = model.deploy(instance_type='ml.g4dn.xlarge', initial_instance_count=1)

In [ ]:
!wget -O /tmp/test.jpg http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0007.jpg
file_name = '/tmp/test.jpg'
# test image
from IPython.display import Image
Image(file_name)  

In [ ]:
import io
import numpy as np
import json
from sagemaker.predictor import StringDeserializer
import time
import cv2
import PIL
from PIL import Image

def numpy_bytes_serializer(data):
    f = io.BytesIO()
    np.save(f, data)
    f.seek(0)
    return f.read()

predictor.serializer = None
predictor.deserializer = StringDeserializer()
predictor.accept = None
predictor.content_type = None

with open(file_name, 'rb') as f:
    payload = f.read()
    #payload = bytearray(payload)
    
im = cv2.imread(file_name, cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)    
is_success, im_buf_arr = cv2.imencode(".jpg", im)
bytes_im = im_buf_arr.tobytes()

for i in range(0, 2):
    response = predictor.predict(bytes_im)            
    print(response)

In [ ]:
#!sudo pip uninstall protobuf
#!sudo pip install -U protobuf
#!pip install -U protobuf

In [ ]:
import io
import numpy as np
import json
from sagemaker.predictor import StringDeserializer
import time
import image_pb2 as impb

with open('/tmp/test.jpg', 'rb') as f:
    payload = f.read()
    #payload = bytearray(payload)

image_packet = impb.PBImage()
image_packet.image_data = payload

def numpy_bytes_serializer(data):
    f = io.BytesIO()
    np.save(f, data)
    f.seek(0)
    return f.read()

predictor.serializer = None
predictor.deserializer = StringDeserializer()
predictor.accept = None
predictor.content_type = 'application/octet-stream'

for i in range(0, 2):    
    response = predictor.predict(image_packet.SerializeToString())            
    print(response)

In [ ]:
!pwd

In [ ]:
!diff /home/ec2-user/SageMaker/src/sagemaker-mxnet-serving-container/docker/1.6.0/py2/Dockerfile.gpu ./Dockerfile.gpu

In [ ]:
!ls /home/ec2-user/SageMaker/GitHub/SageMaker-Inference-Advanced/n-labs/11-manual-model-load/sv.jpg

In [ ]:
!cp /home/ec2-user/SageMaker/GitHub/SageMaker-Inference-Advanced/n-labs/11-manual-model-load/sv.jpg . 


In [ ]:
from matplotlib.pyplot import imshow
import os
import io
import subprocess
import PIL
from PIL import Image
import requests
import random
import io
import numpy as np
import json
from sagemaker.predictor import StringDeserializer
import time
import image_pb2 as impb
import ast
from IPython.display import clear_output

from sagemaker.predictor import RealTimePredictor, json_deserializer

urls = []

urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0007.jpg')
urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0001.jpg')
urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0005.jpg')
urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0010.jpg')
urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/038.chimp/038_0012.jpg')
urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/042.coffin/042_0002.jpg')
urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/048.conch/048_0008.jpg')
urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/049.cormorant/049_0002.jpg')
urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/078.fried-egg/078_0003.jpg')
urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/094.guitar-pick/094_0005.jpg')
urls.append('http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/095.hamburger/095_0003.jpg')


impath = '/home/ec2-user/SageMaker/dataset/256_ObjectCategoriesimage'
files = os.listdir(impath)

#%matplotlib inline

while True:
    index = random.randrange(0, len(files))
    #print(files[index])
    file_name = impath + '/' + files[index]
    
    try:
        with open(file_name, 'rb') as f:
            payload = f.read()
            #payload = bytearray(payload)
    except:
        pass
        
    image_packet = impb.PBImage()
    image_packet.image_data = payload

    predictor.serializer = None
    predictor.deserializer = StringDeserializer()
    predictor.accept = None
    predictor.content_type = 'application/octet-stream'

    %time response = predictor.predict(image_packet.SerializeToString())            
    
    print('\n'+response+'\n')
    clear_output(wait=True)
    
